Building CNN using Keras

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D # spatial convolution over images
from keras.layers import MaxPooling2D # operation for spatial data
from keras.layers import Dense # densely-connected NN layer
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical

# Local imports
import adjust_path  # Before doing any local imports
from icc.data_loader import DataLoader
from icc.models.alexnet_model import AlexNet
from icc.models.spencer.alexnet.preprocessing import *

In [ ]:
# parameters for model
learning_rate = 0.0001

In [ ]:
# Start by defining a Sequential model.
model = Sequential()

# Next add layers via the `.add()` method

# Conv Layer 1
model.add(Conv2D(filters=64, 
                 kernel_size=(3, 3), 
                 strides=(1, 1), 
                 padding="valid", 
                 input_shape=(75, 75, 3),
                 kernel_initializer="glorot_normal"))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.2))


# Conv Layer 2
model.add(Conv2D(filters=128, 
                 kernel_size=(3, 3), 
                 strides=(1, 1), 
                 padding="valid", 
                 kernel_initializer="glorot_normal"))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

# Conv Layer 2
model.add(Conv2D(filters=128, 
                 kernel_size=(3, 3), 
                 strides=(1, 1), 
                 padding="valid", 
                 kernel_initializer="glorot_normal"))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))


# Conv Layer 1
model.add(Conv2D(filters=64, 
                 kernel_size=(3, 3), 
                 strides=(1, 1), 
                 padding="valid", 
                 kernel_initializer="glorot_normal"))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

# Flatten the data for Fully-connected layers. Does not affect the batch size.
model.add(Flatten())

# Fully-connected layers

# Dense Layer 1
model.add(Dense(units=512))
model.add(Activation('elu'))
model.add(Dropout(0.2))

# Dense Layer 1
model.add(Dense(units=256))
model.add(Activation('elu'))
model.add(Dropout(0.2))

model.add(Dense(2))
model.add(Activation('softmax'))

# Before training we need to configure the learning process.
adam_opt = Adam(lr=learning_rate, epsilon=1e-8)
model.compile(optimizer=adam_opt, loss='binary_crossentropy', metrics=['accuracy'])

# Prints a summary representation of your model.
model.summary()

In [ ]:
# Actual training of the network

# Load data
X, y = DataLoader.load_train()
print("{} samples in X and y".format(len(X)))

# Preprocessing step
prep = Preprocess()
x_train, x_valid, y_train, y_valid = prep._basic_trainset(X, y)

# Convert labels to categorical one-hot encoding
y_onehot_train= to_categorical(y_train, num_classes=2)
y_onehot_valid= to_categorical(y_valid, num_classes=2)

In [ ]:
# callbacks
filepath = 'weights.EPOCH{epoch:02d}-VAL_LOSS{val_loss:.2f}.hdf5'
callbacks = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, mode='auto', period=10)

model_history = model.fit(x_train, 
          y_onehot_train, 
          epochs=50, 
          verbose=2, 
          batch_size=24, 
          validation_data=(x_valid, y_onehot_valid), 
          callbacks=[callbacks])

In [ ]:
score = model.evaluate(x_valid, y_onehot_valid)

In [ ]:
score

In [ ]:
preds = model.predict_proba(x_valid)

In [ ]:
best = 'weights.EPOCH30-VAL_LOSS0.51.hdf5'
model.load_weights(filepath=best)

In [ ]:
score = model.evaluate(x_valid, y_onehot_valid)
print('{} = {}'.format(model.metrics_names[0], score[0]))
print('{} = {}'.format(model.metrics_names[1], score[1]))